In [41]:
with open('2315_possible_answers.txt') as fh:
    answer  = fh.readlines()
answer = [line.rstrip('\n') for line in answer]

with open('2315_possible_answers.txt') as fh:
    answer  = fh.readlines()
allowed = [line.rstrip('\n') for line in answer]

In [433]:
import random
import re
import string

class Wardle(object):

    def __init__(self):
        with open('2315_possible_answers.txt') as fh:
            answer  = fh.readlines()
        self.answer = [line.rstrip('\n') for line in answer]

        #print('{0} words added on answer'.format(len(self.answer)))

        with open('12972_allowed_words.txt') as fh:
            allowed  = fh.readlines()
        self.allowed = [line.rstrip('\n') for line in allowed]
        self.allowed_proba = self.createProb(self.allowed )

        #print('{0} words added on allowed'.format(len(self.allowed)))

        self.word = self.wordleWord()
        self.guess_left = 0


        self.passed = self.allowed
        self.gray = ""
        self.green = set()
        self.yellow = ""

        self.wordScores,self.total =  self.calWordScore(self.allowed)
        
        print("Today's Word is *** {0} ***".format(self.word.upper()))

    def createProb(self,wordList):
        l = len(wordList)
        return [1/l] * l
    
    def calWordScore(self,allowed):
        frequency = { ch : [0] * 5 for ch in string.ascii_lowercase}
        for word in allowed:
            for i in range(0,5):
                frequency[word[i]][i] += 1
        
        words = {}
        maxFreq = [0] * 5
        for ch in frequency:
            for i in range(0,5):
                if maxFreq[i] < frequency[ch][i]:
                    maxFreq[i] = frequency[ch][i]
        total = 0
        for word in allowed:
            score  = 1
            for i in range(0,5):
                ch = word[i]
                score *= 1 + (frequency[ch][i] - maxFreq[i]) ** 2
            words.update({word: score})
            total += score
        return words,total

    def wordProb(self):
        total = self.total
        words = self.wordScores
        allowed = self.passed

        prob = []
        passed = []
        for word in allowed:

            prob.append(1 - (1/(1 + np.exp(-words[word]))))
            #prob.append(1 - words[word]/total)
            passed.append(word)
        
        return prob, passed

    def wordleWord(self):
        return random.choice(self.answer)

    def startGuess(self):
        wght,words = self.wordProb()
        guess = random.choices(words, weights = wght, k= 1)[0]
        #guess = random.choice(list(self.passed))
        self.guess_left += 1
        print("{0}".format(guess.upper()))
        self.output(guess)

    def output(self,word):
        for chG,chW in zip(word, self.word):
            if chG == chW:
                self.green.add((chW,self.word.index(chW)))
            if chG not in self.word:
                self.gray += chG
            else:
                if chW not in word :
                    self.yellow += chG
        #print(self.gray,self.green,self.yellow,sep='\n')

    def check(self,str, pattern):
        if re.search(pattern, str):
            return True
        else:
            return False

    def chrPresent(self,chrList):
        l = len(self.passed)
        allowed = self.passed
        if chrList == "":
            self.passed = allowed
            return None
        pattern = re.compile('[{0}]'.format(chrList))
        passed = []
        for word in allowed:
            if self.check(word, pattern):
                passed.append(word)
        self.passed = passed
        #print(self.yellow)
        #print(100 - round(len(self.passed)/l,3) * 100)

    def chrNotPresent(self,chrList):
        l = len(self.passed)
        allowed = self.passed
        pattern = re.compile('[{0}]'.format(chrList))
        passed = []
        for word in allowed:
            if not self.check(word, pattern):
                passed.append(word)
        self.passed = passed
        #print(self.gray)
        #print(100 - round(len(self.passed)/l,3) * 100)
    
    def location(self,greenList,allowed):
        passed = set()
        if greenList == []:
            return allowed
        else:
            ch, loc = greenList[0]
            for word in allowed:
                if word[loc] == ch:
                    passed.add(word)
            return self.location(greenList[1:],list(passed))
    
    def chrLocation(self,greenList):
        l = len(self.passed)
        greenList = list(greenList)
        self.passed = self.location(greenList,self.passed)
        #print(self.green)
        #print(100 - round(len(self.passed)/l,3) * 100)



In [434]:
game.wordScores

{'aahed': 8391994285686699970,
 'aalii': 2923401023081617138176250,
 'aargh': 46234702183059634541125,
 'aarti': 27366377048900059050250,
 'abaca': 128759700381887448633865625,
 'abaci': 162100927752084110432865625,
 'aback': 163957283028287659978551250,
 'abacs': 11982902303837213125,
 'abaft': 149416411595612513440951250,
 'abaka': 116691340313547503673615625,
 'abamp': 175675178339966821247772500,
 'aband': 75984226759419480089022500,
 'abase': 63527617028854738583151250,
 'abash': 137820184281978125157231250,
 'abask': 146479422195869380606232500,
 'abate': 39553981140119982146601250,
 'abaya': 172704447741290665154506250,
 'abbas': 4160288289027989088912500,
 'abbed': 26043307704868374781922500,
 'abbes': 2649848274232641250,
 'abbey': 18707001369186465800562500,
 'abbot': 73406969902276202199345647065000,
 'abcee': 13797837783654620092223125,
 'abeam': 9156875316509267734911412606250,
 'abear': 6930332748308078881468307162500,
 'abele': 5876989112833885956909041355625,
 'abers': 

In [428]:
count = 0
for i in range(100):
    game =  Wardle()
    while len(game.passed) > 1 and  game.guess_left < 6:
        game.startGuess()


        game.chrLocation(game.green)
        game.chrPresent(game.yellow)
        game.chrNotPresent(game.gray)

        if len(game.passed) == 1:
            #print(game.passed[0].upper())
            #print("competed in {0} trail".format(game.guess_left))
            count += 1
            break

print("accuracy:",count/100)
    

Today's Word is *** HYMEN ***
GOTHS
AAHED
WHEEN
HEBEN
HYMEN
Today's Word is *** HIPPY ***
KIDDY
FILLY
ZINCY
JIMMY
HISSY
Today's Word is *** VOCAL ***
DIVNA
SHAUL
TEPAL
CRAAL
COXAL
VOCAL
Today's Word is *** QUASH ***
ANTAR
GLIAS
SCAFF
ODAHS
QUAKY
QUASH
Today's Word is *** GRAVY ***
SMELL
CHIRR
BURKA
DRYAD
PRATY
Today's Word is *** OTTER ***
IGLOO
OKEHS
ORMER
OFFER
OWNER
OTTER
Today's Word is *** FARCE ***
SCALY
VOEMA
AWARD
CRARE
CRARE
ARGUE
Today's Word is *** CLINK ***
TEGGS
OIDIA
CLIMB
CLICK
CLICK
CLICK
Today's Word is *** SHAKE ***
KORMA
VATUS
KIPES
HECKS
NABKS
KEEKS
Today's Word is *** CHECK ***
GAUNT
SLOPS
CIVIE
CERED
CHEWY
CHEEK
Today's Word is *** GROAN ***
NUKED
BINAL
COTAN
HAZAN
MAYAN
ORGAN
Today's Word is *** UNTIE ***
CELLA
WENTS
GYBED
QUOTE
NIXIE
PENIE
Today's Word is *** PUDGY ***
PIPAS
POUCH
PUFFY
PUDDY
PUDDY
PUDDY
Today's Word is *** KNOLL ***
GAMED
CHOLO
FOOLS
OBOLI
TROLL
KNOLL
Today's Word is *** FILLY ***
TEAMS
YUCKO
BLINY
GYPPY
WILLY
DILLY
Today's Word is *** LEFTY **

In [402]:
game =  Wardle()


Today's Word is *** SCORE ***


In [414]:
game.startGuess()
l = len(game.passed)
game.chrLocation(game.green)
game.chrPresent(game.yellow)
game.chrNotPresent(game.gray)
len(game.passed)/l

SCORE
{('c', 1), ('o', 2), ('s', 0), ('r', 3), ('e', 4)}
sesoeees
tiadhunlmbywp


1.0

['becks',
 'bedes',
 'beefs',
 'beeps',
 'beers',
 'bells',
 'bends',
 'benes',
 'beres',
 'bergs',
 'berks',
 'berms',
 'besee',
 'beses',
 'besom',
 'bezes',
 'bhels',
 'blebs',
 'blees',
 'bless',
 'bleys',
 'blobs',
 'blocs',
 'blogs',
 'blows',
 'blubs',
 'bluds',
 'blues',
 'blurs',
 'blush',
 'bobos',
 'bocks',
 'bodes',
 'boeps',
 'boffs',
 'bogus',
 'bohos',
 'bokes',
 'bokos',
 'bolds',
 'boles',
 'bolls',
 'bolos',
 'bolus',
 'bombs',
 'bonds',
 'bones',
 'bongs',
 'bonks',
 'bonus',
 'boobs',
 'boohs',
 'books',
 'bools',
 'booms',
 'boons',
 'boors',
 'boose',
 'bords',
 'bores',
 'borks',
 'borms',
 'bosks',
 'bosky',
 'bosom',
 'boson',
 'bossy',
 'bosun',
 'bouks',
 'bouns',
 'bouse',
 'bousy',
 'bowes',
 'bowls',
 'bowrs',
 'bowse',
 'boxes',
 'boyfs',
 'boygs',
 'boyos',
 'boysy',
 'bozos',
 'breds',
 'brees',
 'brens',
 'brers',
 'brews',
 'breys',
 'brods',
 'brogs',
 'broos',
 'brose',
 'brosy',
 'brows',
 'brush',
 'brusk',
 'bubus',
 'bucks',
 'budos',
 'buffs',
